All of the code can be run on Kaggle using a GPU T4. It will take around ~35-50 minutes. After importing the notebook into Kaggle, you can configure the session options on the right hand side. Choose GPU T4 x2 as the accelerator and turn on the Internet so that the pretrained `resnet50` model from PyTorch can be downloaded. 

This code will output a file called '20898399_Mao.csv' that has the predictions for the test data from my model. This file was submitted to Kaggle.

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.io import read_image
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_and_preprocess_data(csv_path, img_folder):
    df = pd.read_csv(csv_path)

    # Get ancillary data and targets
    ancillary_data = df.iloc[:, 1:164].values  # All variables but 'id'
    targets = df.iloc[:, -6:].values

    # Get image paths
    img_paths = [os.path.join(img_folder, file) for file in os.listdir(img_folder)]

    return img_paths, ancillary_data, targets

In [ ]:
# Parameters
batch_size = 20
learning_rate = 0.001
decay = 0.0001
num_traits = 6
num_epochs = 10

# Based on https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
class PlantDataset(Dataset):
    def __init__(self, img_paths, ancillary_data, targets=None, transform=None):
        self.img_paths = img_paths
        self.ancillary_data = ancillary_data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        # ancillary = torch.tensor(self.ancillary_data[idx], dtype=torch.float32)

        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return img, target  # , ancillary

        return img

class Model(nn.Module):
    def __init__(self, num_ancillary_features=163):
        super(Model, self).__init__()

        # ResNet50
        self.resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

        features = self.resnet.fc.in_features
        # Replace the last layer so that we have 6 output features
        # Based on this Medium article: https://medium.com/@lucrece.shin/chapter-3-transfer-learning-with-resnet50-from-dataloaders-to-training-seed-of-thought-67aaf83155bc
        self.resnet.fc = nn.Linear(features, num_traits)

    def forward(self, img):
        return self.resnet(img)

def train(model, device, train_loader, optimizer, epoch):
  loss_fn = nn.MSELoss()
  model.train()

  train_loss = 0
  # for img, ancillary, targets in train_loader:
  for img, targets in train_loader:
    img, targets = img.to(device), targets.to(device)
    # img, ancillary, targets = img.to(device), ancillary.to(device), targets.to(device)

    optimizer.zero_grad()
    output = model(img)
    # outputs = model(img, ancillary)
    loss = loss_fn(output, targets)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  train_loss /= len(train_loader)

  print(f"Epoch {epoch+1}/{num_epochs}")
  print(f"Train Loss: {train_loss:.4f}")

def test(model, device, test_loader):
  model.eval()
  predictions = []

  with torch.no_grad():
    # for img, ancilliary, _ in test_loader:
    for img in test_loader:
        # img, ancillary = img.to(device), ancillary.to(device)
        img = img.to(device)
        # output = model(img, ancillary)
        output = model(img)
        # From: https://stackoverflow.com/questions/53467215/convert-pytorch-cuda-tensor-to-numpy-array
        predictions.append(output.data.cpu().numpy())

  predictions = np.concatenate(predictions)
  return predictions

In [ ]:
# Load Data
transform = transforms.Compose([
    transforms.ToTensor(), # Convert PIL Image to Tensor
])

train_imgs, train_ancillary, train_targets = load_and_preprocess_data(
    "/kaggle/input/cs-480-2024-spring/data/train.csv",
    "/kaggle/input/cs-480-2024-spring/data/train_images",
)

training_data = PlantDataset(train_imgs, train_ancillary, train_targets, transform)
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

test_imgs, test_ancillary, _ = load_and_preprocess_data(
    "/kaggle/input/cs-480-2024-spring/data/test.csv",
    "/kaggle/input/cs-480-2024-spring/data/test_images"
)

test_data = PlantDataset(test_imgs, test_ancillary, transform=transform)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Intialize model
model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)

# Start training
for epoch in range(num_epochs):
    train(model, device, train_loader, optimizer, epoch)

# Test it and output predictions into CSV file
predictions = test(model, device, test_loader)

# Create submission file
predictions_df = pd.DataFrame({
    'id': pd.read_csv('/kaggle/input/cs-480-2024-spring/data/test.csv')["id"],
    'X4': predictions[:, 0],
    'X11': predictions[:, 1],
    'X18': predictions[:, 2],
    'X26': predictions[:, 3],
    'X50': predictions[:, 4],
    'X3112': predictions[:, 5]
})

predictions_df.to_csv('20898399_Mao.csv', index=False)

Epoch 1/10
Train Loss: 24206300968.7192
Epoch 2/10
Train Loss: 12729432588.2748
Epoch 3/10
Train Loss: 3170605432.5874
Epoch 4/10
Train Loss: 886845.3305
Epoch 5/10
Train Loss: 875169.5243
Epoch 6/10
Train Loss: 870512.9747
Epoch 7/10
Train Loss: 868128.0026
Epoch 8/10
Train Loss: 865889.6672
Epoch 9/10
Train Loss: 865923.6844
Epoch 10/10
Train Loss: 858672.6913
